In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [2]:
links = []
count = 0
main_link = 'https://taling.me/Home/Search/'
query = '?page={}&cateMain={}'

In [3]:
# 코드가 복잡해서 객체지향으로 구현
class TalingScraper():
    def __init__(self, key, courses): #카테고리ID, 검색된 코스 수를 파라미터로 입력
        self.key = key
        self.pages = (int(courses)//15)+1 #페이지 수 수집
        self.main_link = 'https://taling.me/Home/Search/'
        self.query = '?page={}&cateSub={}'
        
    def scrap_info(self):
        key = self.key
        pages = self.pages
        main_link = self.main_link
        query = self.query
        titles = []
        enrolls = []
        links = []
        for i in range(1, pages+1): #페이지 순회하며 title, link, enroll 정보 수집
            res = requests.get((main_link + query).format(i,key))
            html = res.text
            soup = BeautifulSoup(html,'lxml')
            temp = soup.select('.cont2_box .cont2_class')
            titles.append([title.select_one('div.title').text.strip() for title in temp])
            links.append([main_link+link.a['href'] for link in temp])
            enrolls.append([enroll.select_one('div.d_day').text.strip()[:-4] \
                            if '참여' in enroll.select_one('div.d_day').text else 0\
                            for enroll in temp
                           ])
            print('진행률 = {} / {}'.format(i, pages))
        titles = sum(titles, [])
        links = sum(links, [])
        enrolls = sum(enrolls, [])
        print('링크 수집이 완료되었습니다.')
        return titles, enrolls, links

    def scrap_price(self, titles,links): # 가격 정보는 상세페이지에서 확인할 수 있습니다. 모든 링크를 순회하며 가격 정보를 수집합니다.
        links = links
        titles = titles
        count = 0
        prices_perH = []
        prices = []
        print('도서 수집을 시작합니다.')
        for link in links:
            count += 1
            res = requests.get(link)
            html = res.content
            soup = BeautifulSoup(html,'lxml')
                
            prices_perH.append(soup.select_one('p.price_info > span.per').text[:-6])
            prices.append(soup.select_one('p.price_info > span.total > b').text[:-3])
            print("클레스명 : {} | 진행률 : {}/{}".format(titles[count-1], count, len(links)))
        print('클레스 가격 정보 수집이 완료되었습니다.')
        return prices_perH,prices

    def make_df(self, name, links, titles, enrolls, prices, prices_perH): # 모든 데이터로 데이터프레임을 생성합니다.
        df = pd.DataFrame({'링크': links, '강좌명': titles, '등록수': enrolls, '가격': prices,\
                           '시간당 단가': prices_perH})
        df.to_csv('./{}.csv'.format(name))
        print('도서 리스트를 csv파일로 저장하였습니다. 저장 경로는 현재 실행 위치입니다.')
        return df


In [4]:
Scraper = TalingScraper(11, 52) # test param->(id, 코스 수)

In [5]:
titles, enrolls, links = Scraper.scrap_info() # return된 값을 변수로 받아냅니다.


진행률 = 1 / 4
진행률 = 2 / 4
진행률 = 3 / 4
진행률 = 4 / 4
링크 수집이 완료되었습니다.


데이터 수가 일치하는지(중간에 누락된 값이 없는지) 확인합니다

In [6]:
len(titles)

57

In [7]:
len(links)

57

In [8]:
len(enrolls)

57

In [9]:
prices_perH,prices =Scraper.scrap_price(titles,links) # 링크를 순회하며 가격정보를 추출합니다.

도서 수집을 시작합니다.
클레스명 : [3월모집] 1도 모른 나, 6주 후 실무 쿼리를 작성하다니!   #데이터베이스(SQL) 입문부터 실무까지 | 진행률 : 1/57
클레스명 : [3월 원데이 모집 중] 비전공자도 즉시 써먹는 데이터분석 & 시각화 실습! 'R' | 진행률 : 2/57
클레스명 : [SQL] 쉽게 배우는 실무 데이터분석! 기초부터 현업사례까지! | 진행률 : 3/57
클레스명 : 서울대 출신 경제학 박사가 가르치는 딥러닝 예측방법론 | 진행률 : 4/57
클레스명 : [파워비아이공방/기초반]일반 Excel유저를 위한 Power BI 강의 (by 현직컨설턴트 | 진행률 : 5/57
클레스명 : [2021년 2월3월/정규반 모집중! 입문자 R데이터 분석]  단계적으로 배우는 R 패키지 | 진행률 : 6/57
클레스명 : 엑셀 및 R로 이해하는 통계적 분석 (수강자 맞춤형 진행 가능) | 진행률 : 7/57
클레스명 : 네이버 API 활용하기 + 트렌드 데이터 시각화 | 진행률 : 8/57
클레스명 : Numpy, Pandas 배워서 파이썬 제대로 사용하기 | 진행률 : 9/57
클레스명 : 파이썬 배울 때 이것만은 꼭 알고가자! Py린이를 위한 코딩 입문 | 진행률 : 10/57
클레스명 : 초보자 머신러닝/딥러닝 시작하기 | 진행률 : 11/57
클레스명 : 인공지능 개발을 위한 파이썬 고성능 컴퓨팅 | 진행률 : 12/57
클레스명 : 하루에 1% 수익률! 인공지능 주식 로봇 만들기 | 진행률 : 13/57
클레스명 : [SQL / Tableau / Power BI] 현업에 필요한 데이터 분석 Skill | 진행률 : 14/57
클레스명 : [합격 사업계획서 작성법] 대출X 갚지 않는 정부지원금 받는 정부지원사업으로 리스크 없이 사업하자! 예비창업자 창업자 자영업자 가능 | 진행률 : 15/57
클레스명 : [Online] 비전공자도 즉시 써먹는 데이터분석 & 시각화 실습! 'R | 진행률 : 16/57
클레스명 : [사회과학] 논문 통계/연

In [10]:
df = Scraper.make_df('DataScience', links, titles, enrolls, prices, prices_perH)

도서 리스트를 csv파일로 저장하였습니다. 저장 경로는 현재 실행 위치입니다.


In [ ]:
df.to_csv('./data/taling_scraper.csv')